In [12]:
import sys
import os
modules_path = os.getcwd() + '/src/python'
sys.path.insert(0, modules_path)

# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from notebooks.utils import compute_user_penalty, get_team_values_df
from common.load import load_competition_data, process_team_logs

# Import common data

In [13]:
config = 'src/python/config_vbs2023.yaml'

# load competition data from dres files and auxiliary data (FPSs, sequences)
comp_data = load_competition_data(config)

# compute team logs and put them in the form of dataframes (or load them if already existing)
logs = process_team_logs(config, comp_data, force=False)

teams = comp_data['config']['teams']

# FIXME: badly patch the name of HTW into vibro
teams_adj = teams.copy()
teams_adj = [t if t != 'HTW' else 'vibro' for t in teams_adj]

teams.remove('VideoCLIP')

/disks/workspace/Workspace/VBS-Analysis/VBS23-Post-Hoc-Analysis/src/python/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS23-Post-Hoc-Analysis/src/python/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS23-Post-Hoc-Analysis/src/python/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS23-Post-Hoc-Analysis/src/python/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

# Analysis

## Time Recall Table

In [14]:
def compute_data_for_time_recall_table(comp_data, teams, logs, split_user=False, only_best_user=False, max_records=1000):
    dfs = []
    for team in teams:
        team_df = logs[team].get_events_dataframe().reset_index()
        team_df = get_team_values_df(comp_data, team_df, split_user if not only_best_user else True, max_records)
        dfs.append(team_df)

    total_df = pd.concat(dfs, axis=0).reset_index()

    # keep the results from the best user among the two, for each given (task, team)
    if only_best_user:
        # the best penalty contribution is given by the ranks. If they are equal, then the user that submitted earlier wins
        user_penalty = compute_user_penalty(total_df, max_records)
                    
        total_df['user_penalty'] = user_penalty
        total_df = total_df.loc[total_df.groupby(['team', 'task'])['user_penalty'].idxmin()]
        total_df = total_df.drop(['user_penalty'], axis=1)
    
    return total_df

def build_table(df, teams, dataset='v3c'):
    # drop unuseful columns
    df = df.drop(['time_first_appearance', 'rank_shot_first_appearance', 'time_last_appearance', 'rank_shot_last_appearance', 'time_first_appearance_video', 'rank_video_first_appearance'], axis=1)

    #renaming task
    rename_fun = lambda x: x.replace('vbs23-kis-t', 'T_').replace('vbs23-kis-v-m', 'M_').replace('vbs23-kis-v', 'V_')
    df['task'] = df['task'].apply(rename_fun)
    tasks = df[['task','task_start']].sort_values(by=['task_start'])['task'].unique()
    textual=[t for t in tasks if t.startswith('T')] #used later to order the column
    visual=[t for t in tasks if t.startswith('V')] #used later to order the column
    marine=[t for t in tasks if t.startswith('M')] #used later to order the column
    df.drop(columns='task_start', inplace=True)

    df = df.fillna(-1)
    col=[c for c in df.columns.values.tolist() if c!='team' and c!='task' and c!='user']
    df[col] = df[col].applymap(lambda x: -1 if x< 0 else x)
    # df = df.astype('str')
    df.replace(['-1'], -1, inplace=True)

    #aggregate
    agg_dic={c: (lambda x: x) for c in col}
    agg_dic['time_correct_submission']="min"
    df=df.groupby(['team','task'])[col].agg(agg_dic).reset_index()
    # df.replace('- / -', '-', regex=True, inplace=True)
    # add_second= lambda x: x if x=='-' else x + 's'
    # df['time_correct_submission']=df['time_correct_submission'].apply(add_second)
    # df['time_best_shot']=df['time_best_shot'].apply(add_second)
    # df['rank_shot_margin_5'] = df['rank_shot_margin_5'].apply(add_second)
    # df['time_best_video'] = df['time_best_video'].apply(add_second)
    # df = df[['team', 'task'] + col].reset_index()

    # if 'time_best_shot', 'time_best_video', 'rank_video', 'rank_shot_margin_0 values are all -1, change them to -2 (one line code)
    df.loc[(df['time_best_shot'] == -1) & (df['time_best_video'] == -1) & (df['rank_video'] == -1) & (df['rank_shot_margin_0'] == -1), ['time_best_shot', 'time_best_video', 'rank_video', 'rank_shot_margin_0', 'time_correct_submission']] = -2

    # put ! where no logs are present
    tasks_unique = pd.DataFrame({'task': textual+visual+marine})
    teams_unique = pd.DataFrame({'team': df['team'].unique()})
    task_team = teams_unique.merge(tasks_unique, how='cross')
    df = task_team.merge(df, how='left').fillna('-2')

    df = df.melt(var_name="metric", id_vars=["team", "task"], value_name="value")
    df['unit'] = df['metric'].apply(lambda x: 
                                    'rank' if x.startswith('rank_') else 
                                    'time_cs' if x == 'time_correct_submission' else
                                    'time_s' if x == 'time_best_shot' else
                                    'time_v' if x == 'time_best_video' else None)
    replace_dic = {
        'rank_shot_margin_0': 'correct frame',
        'time_best_shot': 'correct frame',
        'rank_shot_margin_5': 'frame in GT+2x5s',
        'time_best_shot_margin5': 'frame in GT+2x5s',
        'rank_video': 'correct video',
        'time_best_video': 'correct video',
        'time_correct_submission': 'correct submission'
    }
    df['metric']=df['metric'].map(replace_dic)
    df = df.pivot(index=['team', 'metric', 'unit'], columns="task", values="value")
    df=df.fillna('-1')

    # #sorting index desired order
    # level_0=teams #order in the conf file
    # level_1=['correct frame', 'frame in GT+2x5s', 'correct video','correct submission']
    # level_2=['rank','time']
    # df = df.reindex(pd.MultiIndex.from_product([level_0,level_1,level_2]))
    # df.dropna(axis=0, inplace=True)#'correct submission'/rank shluld not be in the index
    # df_margin_5 = df.copy()

    # sorting index desired order
    level_0 = teams  # order in teh conf file
    level_1 = ['correct frame', 'correct video', 'correct submission']
    level_2 = ['rank', 'time_cs', 'time_v', 'time_s']
    df = df.reindex(pd.MultiIndex.from_product([level_0, level_1, level_2]))
    df.dropna(axis=0, inplace=True)  # 'correct submission'/rank shluld not be in the index

    #ordering columns
    if dataset == 'v3c':
        df = df[textual+visual]
    elif dataset == 'mvk':
        df = df[marine]
    else:
        # put all
        df = df[textual+visual+marine]

    # for each column element (T_1, T_2, T_3, T_5, T_6, ...) replace the name by removing the underscore and re-indexing using consecutive integers
    df.columns = df.columns.map(lambda x: x.replace('_', ' '))
    
    task_names = df.columns.tolist()
    dataset_names = ['MVK' if c.startswith('M') else 'V3C' for c in task_names]
    columns = pd.MultiIndex.from_arrays([dataset_names, task_names])
    df.columns = columns

    df = df.astype(int)
    return df

In [15]:
df = compute_data_for_time_recall_table(comp_data, teams, logs)

df.loc[df['team'] == 'HTW', 'team'] = 'vibro'
table = build_table(df, teams_adj, dataset='v3c-mvk')
table
# table.to_csv('v3c_rank_table.csv')

V3C                                     \
                                       T 1  T 2  T 3  T 4  T 5  T 6  T 7  V 1   
vibro      correct frame      rank       1    1   12   13    4   16   -1    1   
                              time_s    28    6  277  140  104   21   -1   37   
           correct video      rank       1    1   12   13    1   16    1    1   
                              time_v    28    6  277  140   79   21   23   37   
           correct submission time_cs   37   26  395  155  150   29  193   44   
VISIONE    correct frame      rank       1    1    1   16    1    1    4    4   
                              time_s    26   65  384   76   88   23   64   16   
           correct video      rank       1    1    1   16    1    1    4    1   
                              time_v    26   59  384   76   88   23   64   16   
           correct submission time_cs   65   76   -1  111  100   52   95   25   
VIREO      correct frame      rank       6  141  100   20  201    5   -1  185   
                              time_s   137  409   53  187   50   24   -1   17   
           correct video      rank       1  141   64   20    6    5    1   50   
                              time_v    20  409  280  187   91   24  176   22   
           correct submission time_cs  166   -1   -1  253  122  311  191   49   
vitrivr-VR correct frame      rank       7   14   -1   -1   -1    4   -1  189   
                              time_s    27   81   -1   -1   -1   34   -1  132   
           correct video      rank       2   14    4   64    3    3    2   62   
                              time_v    85   81   40  395  130   34   39  132   
           correct submission time_cs  213   -1   93   -1  179   53   88  171   
CVHunter   correct frame      rank      38    4    6   63    4    1   74  339   
                              time_s   268   22  317  350   89  102   38  275   
           correct video      rank       1    4    6   40    3    1    2   21   
                              time_v   268   22  317  350   95  102   51  275   
           correct submission time_cs  345  103   -1   -1  106  142   66   -1   
vitrivr    correct frame      rank       4    4    2  289    1   81   22   65   
                              time_s    23   78   32   88  110   26   39   20   
           correct video      rank       2    4    2    7    1   81    1   30   
                              time_v    47   78   32   88  110   26   39   20   
           correct submission time_cs   94  219   -1  115  116   63   50   68   
Verge      correct frame      rank       3   79  638  132   39    3   -1  518   
                              time_s    40  160  289  345  271  131   -1  201   
           correct video      rank       3   79  124  132   14    3   25   17   
                              time_v    40  160  210  345  369  131   27  201   
           correct submission time_cs  281   -1   -1  356   -1  180  115  222   

                                                                MVK            \
                                       V 2  V 3  V 4  V 5  V 6  M 1  M 2  M 3   
vibro      correct frame      rank     660   54   10   22   12    8    4    6   
                              time_s    17   44   21   30  280   16   30   15   
           correct video      rank      19    2   10    1    2    8    1    3   
                              time_v    29   29   21   39  297   16   30   15   
           correct submission time_cs   53   60   26   45   -1   27   46   24   
VISIONE    correct frame      rank      -1    2   24   19  224    6   80   17   
                              time_s    -1   26   18   62  298   23  205   12   
           correct video      rank     106    1   24    1    2    4   80   17   
                              time_v    87   26   18   65  163   23  205   12   
           correct submission time_cs  112   34   25   72   -1   76  254   27   
VIREO      correct frame      rank      -1   40  141   -1   -1  132  

In [16]:
# Nicely render the table in HTML

default_style = [
    #table properties
    dict(selector=" ", 
            props=[("margin","0"),
                ("font-family",'"Helvetica", "Arial", sans-serif'),
                ("border-collapse", "collapse"),
                ("border","none"),
                ("border", "3px solid #ccf")
    ]),

    #background shading
    dict(selector="tbody tr:nth-child(even)",
            props=[("background-color", "#fff")]),
    dict(selector="tbody tr:nth-child(odd)",
            props=[("background-color", "#eee")]),

    #cell spacing
    dict(selector="td", 
            props=[("padding", ".5em"),
            ('border-left', '1px solid gray'), ('border-right', '1px solid gray')]),

    #header cell properties
    dict(selector="th", 
            props=[("font-size", "100%"),
                   ("padding", ".4em"),
                   ("text-align", "left")]),

    dict(selector=".data",
         props=[("text-align", "center")]),

    dict(selector="thead tr th",
         props=[("text-align", "center"),
                ("font-style", "italic")]),

    dict(selector=".row_heading",
         props=[("font-style", "italic")]),

    # {'selector': 'tbody tr:nth-of-type(1)', 'props': [('border-top', '2px solid gray')]},
    {'selector': 'tbody tr', 'props': [('border-top', '1px solid gray')]},
]

index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal; '
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #262877; color: white;'
}

# Define a custom function to apply formatting
def highlight_cells(x):
    # Create an empty DataFrame with the same shape and index
    styles = pd.DataFrame('', index=x.index, columns=x.columns)

    # Iterate over the DataFrame cells
    for row in range(len(x.index)):
        for col in range(len(x.columns)):
            cell = x.iloc[row, col]
            cell_index = x.index[row]
            cell_column = x.columns[col]

            # Check the conditions for highlighting and apply the appropriate style
            if cell_index[2] == 'rank':
                # correct video but incorrect frame
                if cell_index[1] == 'correct video' and cell > 0 and x.iloc[row-2, col] < 0:
                    styles.iloc[row, col] = 'color: red'

                if cell_index[1] == 'correct frame' and cell < 50 and cell > 0:
                    styles.iloc[row, col] = 'background-color: green'
                elif cell_index[1] == 'correct video' and cell < 10 and cell > 0:
                    styles.iloc[row, col] = 'background-color: yellow'
            if cell_index[1] == 'correct submission':
                if cell == -1 and x.iloc[row-4, col] > 0:
                    # no correct submission but correct frame
                    styles.iloc[row, col] = 'background-color: red'
                elif cell == -1 and x.iloc[row-4, col] == -1 and x.iloc[row-2, col] > 0:
                    # no correct submission but correct video and not correct frame
                    styles.iloc[row, col] = 'background-color: orange'
            if cell == -2:
                styles.iloc[row, col] = 'background-color: lightgray'

    # Return the DataFrame with CSS styles
    return styles

table_copy = table.copy()
table_copy = table_copy.rename(columns={'M 7': 'M 6'})  # FIXME: this is hardcoded, remove it

# Apply the default style to the table
styled_df = table_copy.style.set_table_styles([*default_style, index_names, headers])

styled_df = styled_df.set_table_styles({
    ('MVK', 'M 1'): [{'selector': 'th', 'props': 'border-left: 1px solid lightgray'},
                     {'selector': 'td', 'props': 'border-left: 2px solid gray'}]
}, overwrite=False, axis=0)

styled_df = styled_df.set_table_styles({
    ('V3C', 'T 1'): [{'selector': 'th', 'props': 'border-left: 1px solid lightgray'},
                     {'selector': 'td', 'props': 'border-left: 2px solid gray'}]
}, overwrite=False, axis=0)

# styled_df = styled_df.applymap_index(lambda x: 'text-align:left', axis="index")

styled_df = styled_df.set_properties(subset=pd.IndexSlice[:, "correct frame", "rank"], **{'border-top': '2px solid gray', })

# Apply the formatting using the custom function
styled_df = styled_df.apply(highlight_cells, axis=None)

# Put 's' after the time values
time_rows = styled_df.index.get_level_values(2).str.contains('time')
styled_df.data.loc[time_rows] = styled_df.data.loc[time_rows].applymap(lambda x: f"{x}s" if isinstance(x, (int, float)) and x > 0 else x)

# Render to HTML and postprocess
html = styled_df.to_html()
html = "<html><body>" + html + "</body></html>"

# Some visualization improvements
html = html.replace('-1', '-').replace('-2', '')
html = html.replace('correct frame', 'correct frame/shot')
html = html.replace('time_s', 't<sub>f</sub>').replace('time_v', 't<sub>v</sub>').replace('time_cs', 't<sub>cs</sub>')

with open('plots/rank_table.html', 'w') as f:
    f.write(html)

In [20]:
# exclude videoclip and VERGE that have not logged MVK
df_filtered = df[~df['team'].isin(['VideoCLIP', 'Verge'])]

df_filtered.loc[df_filtered['team'] == 'HTW', 'team'] = 'vibro'
table = build_table(df_filtered, teams_adj, dataset='mvk')
table
# table.to_csv('mvk_rank_table.csv')

MVK                        
                                       M 1  M 2  M 3  M 4  M 5 M 7
vibro      correct frame      rank       8    4    6    5   41   3
                              time_s    16   30   15   94   23  11
           correct video      rank       8    1    3    5   41   1
                              time_v    16   30   15   94   23  11
           correct submission time_cs   27   46   24  211   53  23
VISIONE    correct frame      rank       6   80   17   33   27   2
                              time_s    23  205   12  245   19  37
           correct video      rank       4   80   17   33   27   1
                              time_v    23  205   12  245   19  12
           correct submission time_cs   76  254   27  266   51  44
VIREO      correct frame      rank     132   78  216  188  198   5
                              time_s    16   20   16  101  219   7
           correct video      rank      26   78   40  188  198   4
                              time_v    16   20   72  101  219   7
           correct submission time_cs   43  110  101  183   -1  11
vitrivr-VR correct frame      rank     105   12   44   -1  546   1
                              time_s    13  161  160   -1   66  16
           correct video      rank      99   12   44  171  546   1
                              time_v    15  161  160  215   66  16
           correct submission time_cs   38  270   62  265  102  26
CVHunter   correct frame      rank      10   13    1  359    1   9
                              time_s    98   67   20  241   31  10
           correct video      rank       1    1    1  148    1   9
                              time_v    98   67   20  255   31  10
           correct submission time_cs  111   83   28   -1   39  14
vitrivr    correct frame      rank     200   24    6   -1   16   1
                              time_s    15  201  179   -1   38  16
           correct video      rank      46   24    6  641   16   1
                              time_v    21  201  179  219   38  16
           correct submission time_cs   80   -1  196   -1   56  22